In [2]:
!pip install opensmile

  Using cached opensmile-2.5.0-py3-none-win_amd64.whl.metadata (15 kB)
  Using cached audobject-0.7.11-py3-none-any.whl.metadata (2.6 kB)
  Using cached audinterface-1.2.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached audeer-2.0.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached audformat-1.1.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached audiofile-1.4.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached audmath-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached audresample-1.3.3-py3-none-win_amd64.whl.metadata (4.4 kB)
  Using cached oyaml-1.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached iso_639-0.4.5-py3-none-any.whl
  Using cached iso3166-2.1.1-py3-none-any.whl.metadata (6.6 kB)
Using cached opensmile-2.5.0-py3-none-win_amd64.whl (793 kB)
Using cached audinterface-1.2.1-py3-none-any.whl (66 kB)
Using cached audobject-0.7.11-py3-none-any.whl (43 kB)
Using cached audeer-2.0.0-py3-none-any.whl (39 kB)
Using cached audformat-1.1.4-py3-none-any.whl (141 kB)
Using c

In [35]:
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
import pandas as pd
import opensmile

In [4]:
df = pd.DataFrame({ 'id': [fn[:3] for fn in os.listdir('./dataset') if fn.endswith('.wav')] })
df.head()

,id
0,300
1,301
2,302
3,303
4,304


In [5]:
is_depressed = {}
for fn in [
    'dev_split_Depression_AVEC2017.csv',
    'train_split_Depression_AVEC2017.csv',
    'test_split_Depression_AVEC2017.csv',
    'full_test_split.csv']:
    with open('./dataset/'+fn, 'r') as f:
        for row in f.readlines()[1:]:
            vals = row[:-1].split(',')
            if len(vals) >= 2:
                is_depressed[vals[0]] = int(vals[1])
df['is_depressed'] = df['id'].apply(lambda id: is_depressed[id] if id in is_depressed else 0)
df.head()

,id,is_depressed
0,300,0
1,301,0
2,302,0
3,303,0
4,304,0


In [33]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
smile_df = smile.process_files([f'./dataset/{id}_AUDIO.wav' for id in df['id'].tolist()])
smile_df.head()

,,,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
file,start,end,,,,,,,,,,,,,,,,,,,,,
./dataset/300_AUDIO.wav,0 days,0 days 00:10:48.500000,2.835027,0.331277,0.016178,0.073676,0.092670,0.169379,0.018994,0.076709,0.095703,0.060193,...,8.667942,0.601993,1.848118,1.848170,-20.0,0.581762,89.249535,44.508762,88.691612,44.451393
./dataset/301_AUDIO.wav,0 days,0 days 00:13:43.900000,1.563116,0.992086,0.023573,0.065601,0.094496,0.226772,0.028895,0.132276,0.161170,0.059034,...,9.814976,0.585230,2.147391,2.147398,-20.0,0.601254,95.877174,49.022385,94.419037,48.620708
./dataset/302_AUDIO.wav,0 days,0 days 00:12:38.800000,0.844108,0.954160,0.000699,0.061128,0.065454,0.101554,0.004327,0.036100,0.040426,0.056394,...,9.711348,0.589838,1.997163,1.997292,-20.0,0.609909,93.560135,47.109814,93.347351,46.971210
./dataset/303_AUDIO.wav,0 days,0 days 00:16:25.300000,1.446217,0.840322,0.039026,0.062715,0.098875,0.221331,0.036160,0.122456,0.158616,0.054199,...,11.710479,0.557672,2.333878,2.333952,-20.0,0.636285,97.992241,51.536892,97.227127,51.981255
./dataset/304_AUDIO.wav,0 days,0 days 00:13:12.600000,1.083328,0.410899,0.023747,0.057617,0.074131,0.142586,0.016514,0.068455,0.084969,0.051853,...,10.999923,0.598984,2.158344,2.158308,20.0,0.548527,97.093452,48.878418,95.413612,50.013947


In [101]:
#smile_df = smile_df.drop('file', axis=1)
X_train, X_test, y_train, y_test = train_test_split(smile_df, df['is_depressed'], stratify=df['is_depressed'])

inp = layers.Input(shape=(6373))
out = layers.Dense(128, activation="relu")(inp)
out = layers.Dense(64, activation="relu")(out)
out = layers.Dense(32, activation="relu")(out)
out = layers.Dense(1, activation='sigmoid')(out)
model = Model(inputs=inp, outputs=out)

optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 6373)]            0         
                                                                 
 dense_26 (Dense)            (None, 128)               815872    
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 826,241
Trainable params: 826,241
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(x=X_train, y=y_train, epochs=50)

Epoch 1/50
5/5 [==============================] - 1s 9ms/step - loss: 203960256.0000
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 11722199.0000
Epoch 3/50
5/5 [==============================] - 0s 10ms/step - loss: 30550756.0000
Epoch 4/50
5/5 [==============================] - 0s 15ms/step - loss: 12953893.0000
Epoch 5/50
5/5 [==============================] - 0s 11ms/step - loss: 19323828.0000
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 10811084.0000
Epoch 7/50
5/5 [==============================] - 0s 10ms/step - loss: 36578172.0000
Epoch 8/50
5/5 [==============================] - 0s 10ms/step - loss: 31209686.0000
Epoch 9/50
5/5 [==============================] - 0s 11ms/step - loss: 32568846.0000
Epoch 10/50
5/5 [==============================] - 0s 11ms/step - loss: 17067880.0000
Epoch 11/50
5/5 [==============================] - 0s 10ms/step - loss: 10190513.0000
Epoch 12/50
5/5 [==============================] - 0s 11ms/step

In [103]:
pred = model.predict(X_test)
print(classification_report(y_test, pred))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        32
           1       0.00      0.00      0.00        14

    accuracy                           0.70        46
   macro avg       0.35      0.50      0.41        46
weighted avg       0.48      0.70      0.57        46



C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\denni\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)